# SQL Options in Spark HW

Alirght let's apply what we learned in the lecture to a new dataset!

**But first!**

Let's start with Spark SQL. But first we need to create a Spark Session!

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Spark-SQL-DEMO").getOrCreate()
    


22/01/17 18:23:10 WARN Utils: Your hostname, Legion resolves to a loopback address: 127.0.1.1; using 172.27.192.232 instead (on interface eth0)
22/01/17 18:23:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/01/17 18:23:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Read in our DataFrame for this Notebook

For this notebook we will be using the Google Play Store csv file attached to this lecture. Let's go ahead and read it in. 

### About this dataset

Contains a list of Google Play Store Apps and info about the apps like the category, rating, reviews, size, etc. 

**Source:** https://www.kaggle.com/lava18/google-play-store-apps

In [4]:
spark
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()

print("Cores uses: ", cores)

Cores uses:  1


In [27]:
path = "/home/iron/Documents/1.Learning/2. PYSPARK_BASICS/Read Write Validate Datasets/datasets/"

storeAppsDF = spark.read.csv(path+"googleplaystore.csv", inferSchema=True, header=True)

storeAppsReviewsDF = spark.read.csv(path+"googleplaystore_user_reviews.csv", inferSchema=True, header=True)


## First things first

Let's check out the first few lines of the dataframe to see what we are working with

In [21]:
storeAppsReviewsDF.limit(100).toPandas()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,"""I like eat delicious food. That's I'm cooking...","also """"Best Before (Shelf Life)""""""",Positive,1.0
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.28846153846153844
2,10 Best Foods for You,nan,nan,nan,nan
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.4,0.875
4,10 Best Foods for You,Best idea us,Positive,1.0,0.3
...,...,...,...,...,...
95,10 Best Foods for You,Boring. I thought actually just texts that's i...,Negative,-0.32499999999999996,0.47500000000000003
96,10 Best Foods for You,Love app. It gives way good idea I buying stor...,Positive,0.39999999999999997,0.4000000000000001
97,10 Best Foods for You,Great Its really best unique provides detailed...,Positive,0.5791666666666666,0.6833333333333332
98,10 Best Foods for You,Nothing special! Could find anything useful!,Positive,0.4107142857142857,0.2857142857142857


As well as the schema to make sure all the column types were correctly infered

In [28]:
storeAppsReviewsDF.printSchema()
# Sentiment_Subjectivity - should be double 

root
 |-- App: string (nullable = true)
 |-- Translated_Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)
 |-- Sentiment_Polarity: string (nullable = true)
 |-- Sentiment_Subjectivity: string (nullable = true)



In [29]:
storeAppsDF.limit(5).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [30]:
storeAppsDF.printSchema()

# rating - float
# reviews - int
# Last Updated - date

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



Looks like we need to edit some of the datatypes. We need to update Rating, Reviews and Price as integer (float for Rating) values for now, since the Size and Installs variables will need a bit more cleaning. Since we haven't been over this yet, I'm going to provide the code for you here so you can get a quick look at how it used (and how often we need it!).

**make sure to change the df name to whatever you named your df**

In [31]:
from pyspark.sql.types import IntegerType, FloatType, DateType
# rating, reviews and Price

storeAppsDF = storeAppsDF.withColumn("Rating",storeAppsDF["Rating"].cast(FloatType())) \
                            .withColumn("Reviews", storeAppsDF["Reviews"].cast(IntegerType())) \
                                .withColumn("Price", storeAppsDF["Price"].cast(IntegerType()))
# print(storeAppsDF.printSchema())

storeAppsDF.limit(100).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,All of the parking lot - National Park applica...,AUTO_AND_VEHICLES,4.0,1754,14M,"500,000+",Free,0,Everyone,Auto & Vehicles,"June 2, 2018",2.3.4,4.0 and up
96,Inquiry Fines and Debits of Vehicles,AUTO_AND_VEHICLES,4.4,2680,2.2M,"500,000+",Free,0,Everyone,Auto & Vehicles,"March 20, 2018",1.03,4.0.3 and up
97,Gas Station,AUTO_AND_VEHICLES,4.0,1288,4.5M,"100,000+",Free,0,Everyone,Auto & Vehicles,"April 21, 2018",2.17,4.0 and up
98,Hush - Beauty for Everyone,BEAUTY,4.7,18900,17M,"500,000+",Free,0,Everyone,Beauty,"August 2, 2018",6.10.1,5.0 and up


Looks like that worked! Great! Let's dig in. 

## 1. Create Tempview

Go ahead and create a tempview of the dataframe so we can work with it in spark sql.

In [87]:
storeAppsDF.createOrReplaceTempView("PlayStoreAppsList")

## 2. Select all apps with ratings above 4.1

Use your tempview to select all apps with ratings above 4.1

In [52]:
spark.sql("select * from PlayStoreAppsList where rating > 4.1").limit(10).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
1,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
2,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
3,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
4,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up
5,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1M,"10,000+",Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3 and up
6,Text on Photo - Fonteee,ART_AND_DESIGN,4.4,13880,28M,"1,000,000+",Free,0,Everyone,Art & Design,"October 27, 2017",1.0.4,4.1 and up
7,Name Art Photo Editor - Focus n Filters,ART_AND_DESIGN,4.4,8788,12M,"1,000,000+",Free,0,Everyone,Art & Design,"July 31, 2018",1.0.15,4.0 and up
8,Tattoo Name On My Photo Editor,ART_AND_DESIGN,4.2,44829,20M,"10,000,000+",Free,0,Teen,Art & Design,"April 2, 2018",3.8,4.1 and up
9,Mandala Coloring Book,ART_AND_DESIGN,4.6,4326,21M,"100,000+",Free,0,Everyone,Art & Design,"June 26, 2018",1.0.4,4.4 and up


## 3. Now pass your results to an object 
(ie create a spark dataframe)

Select just the App and Rating column where the Category is in the Comic category and the Rating is above 4.5.

In [63]:
ratingsDF = spark.sql("SELECT App,Rating FROM PlayStoreAppsList WHERE Category = 'COMICS' AND Rating > 4.5")
ratingsDF.limit(5).toPandas()

,App,Rating
0,Manga Master - Best manga & comic reader,4.6
1,GANMA! - All original stories free of charge f...,4.7
2,Röhrich Werner Soundboard,4.7
3,Unicorn Pokez - Color By Number,4.8
4,Manga - read Thai translation,4.6


## 4. Which category has the most cumulative reviews

Only select the one category with the most reivews. 

*Note: will require adding all the review together for each category*

In [67]:
sum_cat_df = spark.sql(" SELECT Category, sum(Reviews) AS ReviewsCount FROM PlayStoreAppsList group by Category order by ReviewsCount desc ")

# result_df = spark.sql(" select max(reviews_count) from sum_cat_df ")

sum_cat_df.show(10)

+---------------+------------+
|       Category|ReviewsCount|
+---------------+------------+
|           GAME|  1585422349|
|  COMMUNICATION|   815462260|
|         SOCIAL|   621241422|
|         FAMILY|   410226330|
|          TOOLS|   273185044|
|    PHOTOGRAPHY|   213516650|
|       SHOPPING|   115041222|
|   PRODUCTIVITY|   114116975|
|  VIDEO_PLAYERS|   110380188|
|PERSONALIZATION|    89346140|
+---------------+------------+
only showing top 10 rows



## 5. Which App has the most reviews?

Display ONLY the top result

Include only the App column and the Reviews column.

In [79]:
max_reviews = spark.sql(" select  App AS App_Name, Reviews AS max_rev from PlayStoreAppsList order by Reviews desc ").show(1)


+--------+--------+
|App_Name| max_rev|
+--------+--------+
|Facebook|78158306|
+--------+--------+
only showing top 1 row



## 5. Select all apps that contain the word 'dating' anywhere in the title

*Note: we did not cover this in the lecture. You'll have to use your SQL knowledge :) Google it if you need to.*

In [83]:
## 5. Which App has the most reviews?
dating_apps = spark.sql( "select App, Category from PlayStoreAppsList where App like '%dating%' " )
dating_apps.show(truncate=False)

+--------------------------------------------------+---------+
|App                                               |Category |
+--------------------------------------------------+---------+
|Meet, chat & date. Free dating app - Chocolate app|DATING   |
|Friend Find: free chat + flirt dating app         |DATING   |
|Spine- The dating app                             |DATING   |
|Princess Closet : Otome games free dating sim     |FAMILY   |
|happn – Local dating app                          |LIFESTYLE|
+--------------------------------------------------+---------+



## 6. Use SQL Transformer to display how many free apps there are in this list

In [92]:
from pyspark.sql.functions import expr
# First we need to import SQL transformer
from pyspark.ml.feature import SQLTransformer

In [95]:
sqlTrans = SQLTransformer(
    statement="SELECT App as Total FROM __THIS__ where Price=0 ") 
sqlTrans.transform(storeAppsDF).show(100, truncate=False)

+------------------------------------------------------------------------------------------------------------+
|Total                                                                                                       |
+------------------------------------------------------------------------------------------------------------+
|Photo Editor & Candy Camera & Grid & ScrapBook                                                              |
|Coloring book moana                                                                                         |
|U Launcher Lite – FREE Live Cool Themes, Hide Apps                                                          |
|Sketch - Draw & Paint                                                                                       |
|Pixel Draw - Number Art Coloring Book                                                                       |
|Paper flowers instructions                                                                                  |
|

## 7. What is the most popular Genre?

Which genre appears most often in the dataframe. Show only the top result.

In [96]:
storeAppsDF.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



## 8. Select all the apps in the 'Tools' genre that have more than 100 reviews

In [101]:
sql_trans = SQLTransformer(
            statement = "select Genres, count(Genres) as Genres_Count from __THIS__ group by Genres order by Genres_Count desc "
)

top_genres = sql_trans.transform(storeAppsDF)

top_genres.show(10, truncate=False)

+---------------+------------+
|Genres         |Genres_Count|
+---------------+------------+
|Tools          |842         |
|Entertainment  |623         |
|Education      |549         |
|Medical        |463         |
|Business       |460         |
|Productivity   |424         |
|Sports         |398         |
|Personalization|392         |
|Communication  |387         |
|Lifestyle      |381         |
+---------------+------------+
only showing top 10 rows



In [108]:
sqlTrans = SQLTransformer(
    statement="SELECT App, Reviews FROM __THIS__ WHERE Genres = 'Tools' AND Reviews > 100") 
sqlTrans.transform(storeAppsDF).show(10)

+--------------------+--------+
|                 App| Reviews|
+--------------------+--------+
|   Moto File Manager|   38655|
|              Google| 8033493|
|    Google Translate| 5745093|
|        Moto Display|   18239|
|      Motorola Alert|   24199|
|     Motorola Assist|   37333|
|Cache Cleaner-DU ...|12759663|
|  Moto Suggestions ™|     308|
|          Moto Voice|   33216|
|          Calculator|   40770|
+--------------------+--------+
only showing top 10 rows



## That's all folks! Great job!